# QGAN

QGAN implemented in PennyLane, Cirq and Tensorflow.

Ref: https://pennylane.ai/qml/demos/tutorial_QGAN.html

In [29]:
! pip install -q pennylane pennylane-cirq

In [1]:
import pennylane as qml
import numpy as np
import tensorflow as tf

In [2]:
dev = qml.device('cirq.simulator', wires=3)

In [3]:
def real(angles, **kwargs):
    qml.Hadamard(wires=0)
    qml.RY(np.pi / 4, wires=1)
    qml.RX(np.pi / 4, wires=0)
    qml.CNOT(wires=[0, 1])
    qml.Rot(*angles, wires=0)
    qml.CNOT(wires=[0, 1])

In [4]:
def generator(w, **kwargs):
    qml.Hadamard(wires=0)
    qml.RX(w[0], wires=0)
    qml.RX(w[1], wires=1)
    qml.RY(w[2], wires=0)
    qml.RY(w[3], wires=1)
    qml.RZ(w[4], wires=0)
    qml.RZ(w[5], wires=1)
    qml.CNOT(wires=[0, 1])
    qml.RX(w[6], wires=0)
    qml.RY(w[7], wires=0)
    qml.RZ(w[8], wires=0)

In [5]:
def discriminator(w):
    qml.Hadamard(wires=0)
    qml.RX(w[0], wires=0)
    qml.RX(w[1], wires=2)
    qml.RY(w[2], wires=0)
    qml.RY(w[3], wires=2)
    qml.RZ(w[4], wires=0)
    qml.RZ(w[5], wires=2)
    qml.CNOT(wires=[0, 2])
    qml.RX(w[6], wires=2)
    qml.RY(w[7], wires=2)
    qml.RZ(w[8], wires=2)

In [6]:
@qml.qnode(dev, interface="tf")
def real_disc_circuit(phi, theta, omega, disc_weights):
    real([phi, theta, omega])
    discriminator(disc_weights)
    return qml.expval(qml.PauliZ(2))

In [7]:
@qml.qnode(dev, interface="tf")
def gen_disc_circuit(gen_weights, disc_weights):
    generator(gen_weights)
    discriminator(disc_weights)
    return qml.expval(qml.PauliZ(2))

In [8]:
def prob_real_true(disc_weights):
    true_disc_output = real_disc_circuit(phi, theta, omega, disc_weights)
    # convert to probability
    prob_real_true = (true_disc_output + 1) / 2
    return prob_real_true


def prob_fake_true(gen_weights, disc_weights):
    fake_disc_output = gen_disc_circuit(gen_weights, disc_weights)
    # convert to probability
    prob_fake_true = (fake_disc_output + 1) / 2
    return prob_fake_true

In [9]:
def disc_cost(gen_weights, disc_weights):
    cost = prob_fake_true(gen_weights, disc_weights) - prob_real_true(disc_weights)
    return cost


def gen_cost(gen_weights, disc_weights):
    return -prob_fake_true(gen_weights, disc_weights)

In [10]:
phi = np.pi / 6
theta = np.pi / 2
omega = np.pi / 7
np.random.seed(0)
eps = 1e-2
init_gen_weights = np.array([np.pi] + [0] * 8) + \
                   np.random.normal(scale=eps, size=(9,))
init_disc_weights = np.random.normal(size=(9,))

gen_weights = tf.Variable(init_gen_weights)
disc_weights = tf.Variable(init_disc_weights)

In [11]:
opt = tf.keras.optimizers.SGD(0.4)

In [12]:
cost = lambda: disc_cost(gen_weights, disc_weights)

for step in range(50):
    opt.minimize(cost, disc_weights)
    if step % 5 == 0:
        cost_val = cost().numpy()
        print("Step {}: cost = {}".format(step, cost_val))

Step 0: cost = -0.03189918398857117
Step 5: cost = -0.1307528018951416
Step 10: cost = -0.2535863071680069
Step 15: cost = -0.3352076858282089
Step 20: cost = -0.3681584820151329
Step 25: cost = -0.3815065249800682
Step 30: cost = -0.3878723606467247
Step 35: cost = -0.39114606380462646
Step 40: cost = -0.39284537732601166
Step 45: cost = -0.393720306456089


In [13]:
print("Prob(real classified as real): ", prob_real_true(disc_weights).numpy())
print("Prob(fake classified as real): ", prob_fake_true(gen_weights, disc_weights).numpy())

Prob(real classified as real):  0.8941125273704529
Prob(fake classified as real):  0.500012144446373


In [14]:
cost = lambda: gen_cost(gen_weights, disc_weights)

for step in range(50):
    opt.minimize(cost, gen_weights)
    if step % 5 == 0:
        cost_val = cost().numpy()
        print("Step {}: cost = {}".format(step, cost_val))

Step 0: cost = -0.5706225112080574
Step 5: cost = -0.8653104603290558
Step 10: cost = -0.9678259789943695
Step 15: cost = -0.9894888401031494
Step 20: cost = -0.9958748519420624
Step 25: cost = -0.9982682466506958
Step 30: cost = -0.9992517828941345
Step 35: cost = -0.9996717572212219
Step 40: cost = -0.9998548626899719
Step 45: cost = -0.9999353289604187


In [15]:
print("Prob(fake classified as real): ", prob_fake_true(gen_weights, disc_weights).numpy())
print("Discriminator cost: ", disc_cost(gen_weights, disc_weights).numpy())

Prob(fake classified as real):  0.9999659955501556
Discriminator cost:  0.10585346817970276


In [16]:
obs = [qml.PauliX(0), qml.PauliY(0), qml.PauliZ(0)]

bloch_vector_real = qml.map(real, obs, dev, interface="tf")
bloch_vector_generator = qml.map(generator, obs, dev, interface="tf")

print("Real Bloch vector: {}".format(bloch_vector_real([phi, theta, omega])))
print("Generator Bloch vector: {}".format(bloch_vector_generator(gen_weights)))

Real Bloch vector: [-0.21694186  0.45048431 -0.61237222]
Generator Bloch vector: [-0.34041026  0.53855956 -0.77069694]
